In [138]:
import jax
import jax.numpy as jnp
import numpy as np
from jax import random

key = random.PRNGKey(80)
random.uniform(key)

DeviceArray(0.750391, dtype=float32)

In [139]:
np.random.seed(1)
N = 4
beta = 1

sigma_z = jnp.array([[1,0],[0,-1]])
sigma_x = jnp.array([[0,1],[1,0]])
sigma_y = jnp.array([[0,1],[1,0]])

# Generate x 

spin_choices = jnp.hstack(([[0],[1]],)*N)
spin = jnp.array([0,1])
spin_up = jnp.array([[1],[0]])
spin_down= jnp.array([[0],[1]])

epsilons = np.random.rand(2,N)
eps = jnp.array(0.5 * np.random.rand(N))
p_down = 1/(1+jnp.exp(-beta * eps))
p_up = jnp.exp(-beta * eps)/(1+jnp.exp(-beta * eps))
probs = jnp.vstack((p_down,p_up))


def choose_spin(prob):
    key = random.split()
    choice = random.choice(key,spin,p=prob)
    up_func = lambda x: spin_up
    down_func = lambda x: spin_down
    if choice == 0:
        return spin_up
    else:
        return spin_down
    return jax.lax.cond(choice,up_func,down_func,0)




rand_spin =  jax.vmap(choose_spin)(probs.T)
rand_spin = rand_spin.T



TypeError: split() missing 1 required positional argument: 'key'

In [140]:
# create H

J = 0.1

H = jnp.zeros((2**N,2**N))

initLattice = jnp.kron(sigma_z,sigma_z)
for i in range(2,N):
    initLattice = jnp.kron(initLattice,jnp.eye(2))

H += - J * initLattice

for lattice_point in range(1,N-1):
    curr = jnp.eye(2)
    for i in range(1,lattice_point):
        curr = jnp.kron(curr,jnp.eye(2))
    curr = jnp.kron( jnp.kron(curr,sigma_z),sigma_z)
    for i in range(lattice_point+2,N):
        curr = jnp.kron(curr,jnp.eye(2))
    
    assert curr.shape[0] == H.shape[0]
    
    H += -J * curr



In [141]:

x0 = jnp.zeros((2**N,1))
x0  = x0.at[0].set(1)
x0.T @ H @ x0

DeviceArray([[-0.3]], dtype=float32)

In [142]:
vec = random.uniform(key,(3,1))
vec

DeviceArray([[0.737398  ],
             [0.5460452 ],
             [0.05000734]], dtype=float32)

It holds that $\det(e^{A}) = e^{Tr(A)}$. The trace of the parametric hermitian H should be vanishing to ensure the unitary has unit det.

In [ ]:
vec = random.uniform(key,(3,1))
jnp.diag(vec[0:1,0],k=-1) + jnp.diag(vec[1:,0])


vec = random.uniform(key,(6,1))
jnp.diag(vec[0:1,0],k=-2) + jnp.diag(vec[1:3,0],k=-1) + jnp.diag(vec[3:,0])

In [159]:
#Converts unitary matrix to a vector of parameters (which parameterize the 
#unitary). This vector effectively parameterizes SU(4).

#Unitary is V = exp(1i*B); B is Hermitian

#jax.lax.scan()
N=2

mat_size = int(2**N)

input = random.uniform(key,(1,mat_size**2-1))

input = input[0,:]

vec = jnp.append(input,-jnp.sum(input[-mat_size+1:]))
print(input[-mat_size+1:])
print(vec)
h_mat = jnp.diag(vec[-mat_size:])


with np.printoptions(precision=3, suppress=True):
    print(h_mat)

assert jnp.trace(h_mat) < 0.00001

count = 0
#real part
for i in range(1,mat_size):
    h_mat += jnp.diag(vec[count:count+i],k=-mat_size+i)
    count += i

#img part
for i in range(1,mat_size):
    h_mat += jnp.diag(1j * vec[count:count+i],k=-mat_size+i)
    count += i

h_mat = (h_mat.conjugate().T + h_mat)/2

assert jnp.trace(h_mat) < 0.000001

unitary = jnp.expm1(1j * h_mat)


with np.printoptions(precision=3, suppress=True):
    print(h_mat)

I = unitary.conjugate().T @ unitary


with np.printoptions(precision=3, suppress=True):
    print(I)
jnp.linalg.det(unitary)

[0.9809257  0.37119985 0.8251778 ]
[ 0.1811248   0.13589895  0.7806554   0.8635284   0.2445842   0.051229
  0.9615238   0.5502498   0.20359385  0.45245695  0.6299883   0.61170137
  0.9809257   0.37119985  0.8251778  -2.1773033 ]
[[ 0.981  0.     0.     0.   ]
 [ 0.     0.371  0.     0.   ]
 [ 0.     0.     0.825  0.   ]
 [ 0.     0.     0.    -2.177]]
[[ 0.981+0.j     0.432-0.226j  0.068-0.275j  0.091-0.481j]
 [ 0.432+0.226j  0.371+0.j     0.122-0.315j  0.39 -0.102j]
 [ 0.068+0.275j  0.122+0.315j  0.825+0.j     0.026-0.306j]
 [ 0.091+0.481j  0.39 +0.102j  0.026+0.306j -2.177+0.j   ]]
[[1.287+0.j    0.668-0.556j 0.11 -0.62j  0.457-0.32j ]
 [0.668+0.556j 0.66 +0.j    0.331-0.381j 0.194+0.296j]
 [0.11 +0.62j  0.331+0.381j 0.977+0.j    0.593+0.112j]
 [0.457+0.32j  0.194-0.296j 0.593-0.112j 3.841+0.j   ]]


DeviceArray(-0.07410514-0.19634284j, dtype=complex64)

In [118]:
h_mat.conjugate().T

DeviceArray([[0.9809257 -0.j        , 0.8635284 -0.45245695j,
              0.13589895-0.9340702j , 0.1811248 -0.9615238j ],
             [0.        -0.j        , 0.37119985-0.j        ,
              0.2445842 -0.6299883j , 0.7806554 -0.20359385j],
             [0.        -0.j        , 0.        -0.j        ,
              0.8251778 -0.j        , 0.051229  -0.61170137j],
             [0.        -0.j        , 0.        -0.j        ,
              0.        -0.j        , 0.879974  -0.j        ]],            dtype=complex64)

In [35]:
vec = np.random.rand(2,N)
spins = vec/ np.vstack((np.linalg.norm(vec,axis=0),)*2)
porb_up = spins[0,:] 
dist = np.prod()

In [25]:
def bloch_to_rho(vec):
    return (jnp.identity(2) + vec[0] * sigma_x + vec[1]  * sigma_y)/2

def bloch_to_rho_prod(blochVecs):

    rho_prod = bloch_to_rho(blochVecs[0,:])
    for i in range(1,blochVecs.shape[-1]):
        rho_prod = jnp.kron(rho_prod,bloch_to_rho(blochVecs[:,i]))
    return rho_prod

bloch_to_rho_prod(np.random.rand(2,10)).shape

(1024, 1024)

In [27]:

Pr = (jnp.identity(2) - sigma_z)/2


rho_prod = (1/(1 + jnp.exp(- beta*eps[0]))) * jnp.expm1(- beta* eps[0] * Pr)

def rho_prod_sacn(carry,eps_r):
    y = (1/(1 + jnp.exp(- beta*eps_r))) * jnp.expm1(- beta* eps_r * Pr)
    return jnp.kron(carry,y),0



for i in range(1,N):
    eps_r = eps[i]
    y = (1/(1 + jnp.exp(- beta*eps_r))) * jnp.expm1(- beta* eps_r * Pr)
    rho_prod = jnp.kron(rho_prod,y)

#jax.lax.scan(rho_prod_sacn,init,eps[1:])
y



DeviceArray([[-0.        , -0.        ],
             [-0.        , -0.00684679]], dtype=float32)

In [26]:
jnp.sum(jnp.abs(rho_prod)>0)

DeviceArray(1, dtype=int32)

In [33]:
np.outer(x,x.T) @ x

array([0.30151134, 0.30151134, 0.        , 0.        , 0.30151134,
       0.30151134, 0.30151134, 0.30151134, 0.30151134, 0.        ,
       0.        , 0.30151134, 0.        , 0.30151134, 0.30151134,
       0.        , 0.        , 0.30151134, 0.        , 0.        ])

In [ ]:
np.random( )